In [ ]:
#---------- General imports
import numpy as np
import healpy as hp
import copy
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.patches import Ellipse
from scipy.optimize import curve_fit
from matplotlib.colors import SymLogNorm
from astropy.coordinates.sky_coordinate import SkyCoord
from scipy.ndimage.filters import gaussian_filter as gaussfilt
from astropy.nddata import Cutout2D
from astropy.wcs import WCS
import astropy.units as u
import astropy.constants as const
from astropy.io import fits
from astropy.cosmology import FlatLambdaCDM
import os

#---------- Imports related to minot and minot submodules to be run directly
from minot.model import Cluster as CM
from minot.ClusterTools import cluster_electron_loss
from minot.ClusterTools import cluster_profile
from minot.ClusterTools import cluster_spectra
from minot.ClusterTools import cluster_global
from minot.ClusterTools import map_tools

#---------- Baseline plotting style
dict_base = {'font.size':        16, 
             'legend.fontsize':  12,
             'xtick.labelsize':  16,
             'ytick.labelsize':  16,
             'axes.labelsize':   16,
             'axes.titlesize':   16,
             'figure.titlesize': 16,    
             'figure.figsize':[8.0, 6.0],
             'figure.subplot.right':0.97,
             'figure.subplot.left':0.15,
             'font.family':'serif',
             'figure.facecolor': 'white',
             'legend.frameon': True}
plt.rcParams.update(dict_base)

In [ ]:
figdir =  '/Users/vdk/ProjectCTA/TestPerseus/Figure'
workdir = '/Users/vdk/ProjectCTA/TestPerseus/Save'

In [ ]:
#---------- Baseline cluster model definition
cluster = CM(redshift=0.02, M500=7e14*u.Msun, output_dir=workdir, silent=True)
cluster.R_truncation = 5000*u.kpc

#---------- Baseline Profile models
cluster.density_gas_model = {'name':'SVM', 'beta':0.6, 'r_c':290*u.kpc, 
                             'n_0':2.3e-3*u.cm**-3, 'r_s':1000*u.kpc, 'gamma':3, 'epsilon':1.7, 'alpha':0.0}
cluster.pressure_gas_model = {'name':'GNFW', 'P_0':2.0e-2*u.keV/u.cm**3,'a':1.5,'b':3.1,'c':0.00,'c500':3.2}
cluster.set_magfield_isodens_scal_param(5*u.uG, scal=0.5)
cluster.set_density_cre1_isodens_scal_param(scal=1)
cluster.set_density_crp_isodens_scal_param(scal=1)

#---------- Spectral model CR
cluster.spectrum_crp_model = {'name':'PowerLaw', 'Index':2.4}
cluster.spectrum_cre1_model = {'name':'ContinuousInjection', 'Index':2.3, 'BreakEnergy':5*u.GeV}
cluster.X_cre1_E = {'X':1e-5, 'R_norm':cluster.R500}
cluster.X_crp_E  = {'X':1e-2, 'R_norm':cluster.R500}
cluster.cre1_loss_model = 'None'
cluster.Eemin = 1*u.MeV
cluster.Eemax = 10*u.PeV
cluster.Epmin = cluster_spectra.pp_pion_kinematic_energy_threshold() * u.GeV
cluster.Epmax = 10*u.PeV

#---------- Sampling
cluster.map_coord = cluster.coord
cluster.map_fov   = 6*u.deg
cluster.map_reso= 30*u.arcsec
cluster.Npt_per_decade_integ = 50

#---------- Provide information: default plots + parameter
# cluster.plot(directory=cluster.output_dir+'/Baseline') # Generating all automatic plots take a bit of time
#cluster.print_param()

In [ ]:
E = np.logspace(-2,6,100)*u.GeV
r = np.logspace(0,4,100)*u.kpc

#----- Spectrum
ctmp = copy.deepcopy(cluster)

plt.figure(0, figsize=(8,6))
# Baseline
g = ctmp.get_gamma_spectrum(E)[1]
ic = ctmp.get_ic_spectrum(E)[1]
plt.loglog(E, (E**2*g).to_value('GeV cm-2 s-1'), linewidth=3, label='Hadronic emission, baseline', color='blue')
# Changing the CRp slope to 2.1
ctmp2 = copy.deepcopy(cluster)
ctmp2.spectrum_crp_model['Index'] = 2.1
g = ctmp2.get_gamma_spectrum(E)[1]
plt.loglog(E, (E**2*g).to_value('GeV cm-2 s-1'), linewidth=1, linestyle='--', label='CRp slope to $\\alpha = 2.1$', color='blue')
# Changing the CRp slope to 2.8
ctmp2 = copy.deepcopy(cluster)
ctmp2.spectrum_crp_model['Index'] = 2.8
g = ctmp2.get_gamma_spectrum(E)[1]
plt.loglog(E, (E**2*g).to_value('GeV cm-2 s-1'), linewidth=1, linestyle=':', label='CRp slope to $\\alpha = 2.8$', color='blue')
# Changing the CRp density profile to isodensity scaling with eta = 0.5
ctmp2 = copy.deepcopy(cluster)
ctmp2.set_density_crp_isodens_scal_param(scal=0.5)
g = ctmp2.get_gamma_spectrum(E)[1]
# Plot
plt.loglog(E, (E**2*g).to_value('GeV cm-2 s-1'), linewidth=1, linestyle='-.', label=r'Isodensity scaling $n_{\rm CRp} \propto n_{\rm gas}^{0.5}$', color='blue')
plt.loglog(E, (E**2*ic).to_value('GeV cm-2 s-1'), linewidth=1, color='grey', label='Inverse Compton')
plt.xlabel('Energy (GeV)')
plt.ylabel(r'$E^2 \frac{dN}{dEdSdt}$ (GeV cm$^{-2}$ s$^{-1}$)')
plt.xlim(1e-2, 1e6)
plt.ylim(1e-12, 2e-10)
plt.legend()
#plt.savefig(figdir+'/gamma_spectra.pdf')

In [ ]:
# Збільшуємо червоне зміщення
cluster = CM(redshift=0.0498, M500=7e14*u.Msun, output_dir=workdir, silent=True)
cluster.R_truncation = 5000*u.kpc

#---------- Baseline Profile models
cluster.density_gas_model = {'name':'SVM', 'beta':0.6, 'r_c':290*u.kpc, 
                             'n_0':2.3e-3*u.cm**-3, 'r_s':1000*u.kpc, 'gamma':3, 'epsilon':1.7, 'alpha':0.0}
cluster.pressure_gas_model = {'name':'GNFW', 'P_0':2.0e-2*u.keV/u.cm**3,'a':1.5,'b':3.1,'c':0.00,'c500':3.2}
cluster.set_magfield_isodens_scal_param(5*u.uG, scal=0.5)
cluster.set_density_cre1_isodens_scal_param(scal=1)
cluster.set_density_crp_isodens_scal_param(scal=1)

#---------- Spectral model CR
cluster.spectrum_crp_model = {'name':'PowerLaw', 'Index':2.4}
cluster.spectrum_cre1_model = {'name':'ContinuousInjection', 'Index':2.3, 'BreakEnergy':5*u.GeV}
cluster.X_cre1_E = {'X':1e-5, 'R_norm':cluster.R500}
cluster.X_crp_E  = {'X':1e-2, 'R_norm':cluster.R500}
cluster.cre1_loss_model = 'None'
cluster.Eemin = 1*u.MeV
cluster.Eemax = 10*u.PeV
cluster.Epmin = cluster_spectra.pp_pion_kinematic_energy_threshold() * u.GeV
cluster.Epmax = 10*u.PeV

#---------- Sampling
cluster.map_coord = cluster.coord
cluster.map_fov   = 6*u.deg
cluster.map_reso= 30*u.arcsec
cluster.Npt_per_decade_integ = 50

#---------- Provide information: default plots + parameter
# cluster.plot(directory=cluster.output_dir+'/Baseline') # Generating all automatic plots take a bit of time
#cluster.print_param()

In [ ]:
E = np.logspace(-2,6,100)*u.GeV
r = np.logspace(0,4,100)*u.kpc

#----- Spectrum
ctmp = copy.deepcopy(cluster)

plt.figure(0, figsize=(8,6))
# Baseline
g = ctmp.get_gamma_spectrum(E)[1]
ic = ctmp.get_ic_spectrum(E)[1]
plt.loglog(E, (E**2*g).to_value('GeV cm-2 s-1'), linewidth=3, label='Hadronic emission, baseline', color='blue')
# Changing the CRp slope to 2.1
ctmp2 = copy.deepcopy(cluster)
ctmp2.spectrum_crp_model['Index'] = 2.1
g = ctmp2.get_gamma_spectrum(E)[1]
plt.loglog(E, (E**2*g).to_value('GeV cm-2 s-1'), linewidth=1, linestyle='--', label='CRp slope to $\\alpha = 2.1$', color='blue')
# Changing the CRp slope to 2.8
ctmp2 = copy.deepcopy(cluster)
ctmp2.spectrum_crp_model['Index'] = 2.8
g = ctmp2.get_gamma_spectrum(E)[1]
plt.loglog(E, (E**2*g).to_value('GeV cm-2 s-1'), linewidth=1, linestyle=':', label='CRp slope to $\\alpha = 2.8$', color='blue')
# Changing the CRp density profile to isodensity scaling with eta = 0.5
ctmp2 = copy.deepcopy(cluster)
ctmp2.set_density_crp_isodens_scal_param(scal=0.5)
g = ctmp2.get_gamma_spectrum(E)[1]
# Plot
plt.loglog(E, (E**2*g).to_value('GeV cm-2 s-1'), linewidth=1, linestyle='-.', label=r'Isodensity scaling $n_{\rm CRp} \propto n_{\rm gas}^{0.5}$', color='blue')
plt.loglog(E, (E**2*ic).to_value('GeV cm-2 s-1'), linewidth=1, color='grey', label='Inverse Compton')
plt.xlabel('Energy (GeV)')
plt.ylabel(r'$E^2 \frac{dN}{dEdSdt}$ (GeV cm$^{-2}$ s$^{-1}$)')
plt.xlim(1e-2, 1e6)
plt.ylim(1e-12, 2e-10)
plt.legend()
#plt.savefig(figdir+'/gamma_spectra.pdf')

In [ ]:
plt.figure(figsize=(10,8))
r, x_r = clust2.get_crp_to_thermal_energy_profile(Emin=None, Emax=None)
plt.plot(r, x_r, label='Cosmic ray to thermal energy')
plt.vlines(clust2.X_crp_E['R_norm'].to_value('kpc'), clust2.X_crp_E['X']*0.9, clust2.X_crp_E['X']*1.1, label='Imposed normalization')
plt.hlines(clust2.X_crp_E['X'], clust2.X_crp_E['R_norm'].to_value('kpc')*0.9, clust2.X_crp_E['R_norm'].to_value('kpc')*1.1)
plt.xscale('log')
plt.xlabel('radius (kpc)')
plt.ylabel('$U_{CRp}/U_{th} (<r)$')
plt.legend()

In [ ]:
r1, p1 = clust.get_gamma_profile(radius=np.logspace(0,4,100)*u.kpc, 
                                   Emin=500*u.GeV, Emax=1*u.PeV, Energy_density=False)
r2, p2 = clust.get_gamma_profile(radius=np.logspace(0,4,100)*u.kpc, 
                                   Emin=1000*u.GeV, Emax=1*u.PeV, Energy_density=False)
plt.figure(0,figsize=(10,8))
plt.loglog((r1/clust.D_ang*u.rad).to('deg'),p1.to('cm-2 s-1 sr-1'), '-', label='500 GeV - 1 PeV')
plt.loglog((r2/clust.D_ang*u.rad).to('deg'),p2.to('cm-2 s-1 sr-1'), '--', label='1 TeV - 1 PeV')
plt.xlabel('Angular size (deg)')
plt.ylabel('Surface brightness ($ph/cm^2/s/sr$)')
plt.legend()

In [ ]:
energy = np.logspace(-1,4,100)*u.GeV
e1, s1 = clust.get_gamma_spectrum(energy,Rmax=clust.R_truncation, type_integral='spherical')
e2, s2 = clust.get_gamma_spectrum(energy,Rmax=clust.R500, type_integral='spherical')
e3, s3 = clust.get_gamma_spectrum(energy,Rmax=clust.R500, type_integral='cylindrical')

plt.figure(0,figsize=(10,8))
plt.loglog(e1, (e1**2*s1).to('MeV cm-2 s-1'), '-', label='Total volume')
plt.loglog(e2, (e2**2*s2).to('MeV cm-2 s-1'), '-', label='<R500, spherical integration')
plt.loglog(e3, (e3**2*s3).to('MeV cm-2 s-1'), '-', label='<R500, cylindrical integration')
plt.xlabel('Energy (GeV)')
plt.ylabel('Flux ($MeV/cm^2/s$)')
plt.legend()

In [ ]:
energy = np.logspace(-1,4,100)*u.GeV
e1, s1 = clust2.get_gamma_spectrum(energy,Rmax=clust.R_truncation, type_integral='spherical')
e2, s2 = clust2.get_gamma_spectrum(energy,Rmax=clust.R500, type_integral='spherical')
e3, s3 = clust2.get_gamma_spectrum(energy,Rmax=clust.R500, type_integral='cylindrical')

plt.figure(0,figsize=(10,8))
plt.loglog(e1, (e1**2*s1).to('MeV cm-2 s-1'), '-', label='Total volume')
plt.loglog(e2, (e2**2*s2).to('MeV cm-2 s-1'), '-', label='<R500, spherical integration')
plt.loglog(e3, (e3**2*s3).to('MeV cm-2 s-1'), '-', label='<R500, cylindrical integration')
plt.xlabel('Energy (GeV)')
plt.ylabel('Flux ($MeV/cm^2/s$)')
plt.legend()

In [ ]:
# Defines the grid
clust.map_reso = 0.01*u.deg
clust.map_fov = 5.0*u.deg

# The center of the map can be offset wrt the cluster
clust.map_coord = SkyCoord("194.5d  27.2d", frame="icrs", unit="deg")

# Get the corresponding header
header = clust.get_map_header()

In [ ]:
template = clust.get_gamma_map(Emin=None, Emax=None, 
                               Energy_density=False, 
                               Normalize=True) # The map is normalized to the total flux, to be in unit of sr-1

In [ ]:
rad500 = clust.theta500.to_value('deg')
radtru = clust.theta_truncation.to_value('deg')

fig = plt.figure(figsize=(10, 8))
ax = plt.subplot(111, projection=WCS(header))
plt.imshow(template, vmin=0, origin='lower', cmap='magma', norm=SymLogNorm(1))

# Show R500
circle500 = matplotlib.patches.Ellipse((clust.coord.ra.value, clust.coord.dec.value),
                                        2*rad500/np.cos(clust.coord.dec.value*np.pi/180), 2*rad500,
                                        linewidth=2, fill=False, zorder=2,
                                        edgecolor='green', linestyle='dashed',facecolor='none',
                                        transform=ax.get_transform('fk5'))
# Show R_truncation
circletru = matplotlib.patches.Ellipse((clust.coord.ra.value, clust.coord.dec.value),
                                        2*radtru/np.cos(clust.coord.dec.value*np.pi/180), 2*radtru,
                                        linewidth=1, fill=False, zorder=2,
                                        edgecolor='green', linestyle='dashed',facecolor='none',
                                        transform=ax.get_transform('fk5'))
ax.add_patch(circle500)
ax.add_patch(circletru)
plt.colorbar()
plt.title(clust.name+', gamma template ($sr^{-1}$)')

In [ ]:
# Save the parameters
clust.save_param()

# Save relevant output products
clust.save_spectra()
clust.save_profile()
clust.save_map()

# Can also produce plots
clust.plot(prod_list=['all'])

In [ ]:
# Defines the grid
clust2.map_reso = 0.01*u.deg
clust2.map_fov = 5.0*u.deg

# The center of the map can be offset wrt the cluster
clust2.map_coord = SkyCoord("194.5d  27.2d", frame="icrs", unit="deg")

# Get the corresponding header
header = clust2.get_map_header()

template = clust2.get_gamma_map(Emin=None, Emax=None, 
                               Energy_density=False, 
                               Normalize=True) # The map is normalized to the total flux, to be in unit of sr-1

rad500 = clust2.theta500.to_value('deg')
radtru = clust2.theta_truncation.to_value('deg')

fig = plt.figure(figsize=(10, 8))
ax = plt.subplot(111, projection=WCS(header))
plt.imshow(template, vmin=0, origin='lower', cmap='magma', norm=SymLogNorm(1))

# Show R500
circle500 = matplotlib.patches.Ellipse((clust2.coord.ra.value, clust2.coord.dec.value),
                                        2*rad500/np.cos(clust2.coord.dec.value*np.pi/180), 2*rad500,
                                        linewidth=2, fill=False, zorder=2,
                                        edgecolor='green', linestyle='dashed',facecolor='none',
                                        transform=ax.get_transform('fk5'))
# Show R_truncation
circletru = matplotlib.patches.Ellipse((clust2.coord.ra.value, clust2.coord.dec.value),
                                        2*radtru/np.cos(clust2.coord.dec.value*np.pi/180), 2*radtru,
                                        linewidth=1, fill=False, zorder=2,
                                        edgecolor='green', linestyle='dashed',facecolor='none',
                                        transform=ax.get_transform('fk5'))
ax.add_patch(circle500)
ax.add_patch(circletru)
plt.colorbar()
plt.title(clust2.name+', gamma template ($sr^{-1}$)')